In [43]:
import backtrader as bt
import backtrader.feeds as btfeeds

import datetime
import pandas as pd

import schwabdev

import datetime
from dotenv import load_dotenv
import os

In [44]:
load_dotenv()

app_key = os.getenv("APP_KEY")
app_secret = os.getenv("APP_SECRET")

client = client = schwabdev.Client(app_key, app_secret)
data = client.price_history(symbol='NVDA', periodType='year', period=10, frequencyType='daily').json()['candles']
df = pd.DataFrame(data)

df['datetime'] = pd.to_datetime(df['datetime'], unit='ms').dt.strftime('%m-%d-%Y')
df.set_index(inplace=True, keys="datetime")

df.to_csv("data.csv", index=True)

In [45]:
class bh_strat(bt.Strategy):
    params = (
        ('printlog', True),
    )

    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f"{dt.isoformat()} | {txt}")
    
    def __init__(self):
        self.dataclose = self.data.close

        self.order = None
        self.buycomm = None
        self.buyprice = None

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f"BUY EXECUTED | Price: {order.executed.price:.2f} | Cost: {order.executed.value:.2f} | Comm: {order.executed.comm:.2f}")
                
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f"SELL EXECUTED | Price: {order.executed.price:.2f} | Cost: {order.executed.value:.2f} | Comm: {order.executed.comm:.2f}")
        elif order.status in [order.Cancelled, order.Margin, order.Rejected]:
            self.log("ORDER CANCELED / MARGIN / REJECTED")
        
        self.order = None

    def next(self):
        self.log(f"Close: ${self.dataclose[0]:.2f}")

        if len(self) == 1:
            self.log(f"BUY CREATED | Price: {self.dataclose[0]:.2f}")
            self.order = self.buy()
            
    def stop(self):
        self.log(f'Ending Value: {self.broker.getvalue():.2f}', doprint=True)


In [46]:
cerebro = bt.Cerebro()

cerebro.addstrategy(bh_strat)
cerebro.addsizer(bt.sizers.SizerFix, stake=20)
cerebro.broker.setcommission(commission=0.001)

data = bt.feeds.GenericCSVData(
    dataname='data.csv',
    dtformat='%m-%d-%Y',
    datetime=0,           # Column index for date
    open=1,               # Adjust these indices (0, 1, 2...) 
    high=2,               # to match the actual order 
    low=3,                # in your CSV file
    close=4,
    volume=5,
    openinterest=-1,      # -1 means this column doesn't exist
    headers=True          # Set to True if your CSV has a header row
)
cerebro.adddata(data)
cerebro.broker.setcash(10000.0)

In [47]:
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()

print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 10000.00
2016-02-09 | Close: $0.64
2016-02-09 | BUY CREATED | Price: 0.64
2016-02-10 | BUY EXECUTED | Price: 0.64 | Cost: 12.82 | Comm: 0.01
2016-02-10 | Close: $0.64
2016-02-11 | Close: $0.63
2016-02-12 | Close: $0.64
2016-02-16 | Close: $0.67
2016-02-17 | Close: $0.69
2016-02-18 | Close: $0.75
2016-02-19 | Close: $0.76
2016-02-22 | Close: $0.79
2016-02-23 | Close: $0.79
2016-02-24 | Close: $0.80
2016-02-25 | Close: $0.80
2016-02-26 | Close: $0.79
2016-02-29 | Close: $0.78
2016-03-01 | Close: $0.82
2016-03-02 | Close: $0.82
2016-03-03 | Close: $0.82
2016-03-04 | Close: $0.81
2016-03-07 | Close: $0.81
2016-03-08 | Close: $0.79
2016-03-09 | Close: $0.79
2016-03-10 | Close: $0.79
2016-03-11 | Close: $0.81
2016-03-14 | Close: $0.81
2016-03-15 | Close: $0.80
2016-03-16 | Close: $0.83
2016-03-17 | Close: $0.82
2016-03-18 | Close: $0.85
2016-03-21 | Close: $0.85
2016-03-22 | Close: $0.85
2016-03-23 | Close: $0.86
2016-03-24 | Close: $0.86
2016-03-28 | Close: $0.87
2